# Escalamiento de los datos

Lo que el profesor está haciendo con la transformación de features se llama “interacción de variables” y en ocasiones eso genera mejores resultados en una regresión. Por ejemplo la venta de una casa depende del precio, pero también depende de su tamaño, entonces la interacción entre precio y tamaño suele expresarse como una multiplicación entre los valores de cada observación y ese nuevo resultado se vuelve una variable de interacción.

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Diversios algoritmos son sensibles a la escala en la que viene cada feature, **Re-escalarlos** puede tener significativas mejoras de rendimiento

In [3]:
from sklearn.model_selection import train_test_split

X=pd.read_csv('./X.csv')
y = X['worldwide_gross']
X = X.drop('worldwide_gross',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [4]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [5]:
#promedio de features
scaler.mean_

array([3.16539328e+07, 2.00238406e+03, 2.15353335e+00, 1.07623566e+02,
       3.51205842e+07, 6.42548662e+00, 4.87678621e+07])

In [6]:
#desviacion estandard de cada feature
scaler.scale_

array([4.12049618e+07, 1.23960237e+01, 9.79325700e-01, 2.28191442e+01,
       5.84159239e+07, 1.09815209e+00, 6.33824617e+07])

In [7]:
#valores centrados en 0 y en escalas parecidas
scaler.transform(X_train)

array([[-0.69103165, -4.54856033, -0.80007433, ..., -0.54677872,
         1.98015684, -0.00842686],
       [-0.28282839,  0.45304355,  0.20061421, ..., -0.39579249,
        -1.93551206, -0.65895424],
       [-0.7439379 , -0.1923247 , -0.30994117, ..., -0.58409731,
        -0.38745691, -0.76921699],
       ...,
       [-0.404173  ,  0.93706973,  0.20061421, ..., -0.34443663,
         0.61422583, -0.17181256],
       [ 0.44523927,  0.13035942, -0.30994117, ...,  0.25471506,
        -0.38745691, -0.17799441],
       [-0.76213959, -0.11165367, -0.30994117, ..., -0.59693628,
         0.4321017 , -0.70336383]])

In [8]:
X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))

In [9]:
from sklearn.linear_model import Lasso
model = Lasso()
model_scaled = Lasso()

model.fit(X_train, y_train)
model_scaled.fit(X_train_scaled,y_train)

C:\Users\chris\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.349e+18, tolerance: 1.001e+16
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [10]:
print(model.score(X_test,y_test))
print(model_scaled.score(X_test_scaled,y_test))

0.840434269628989
0.8404343154757754


Los modelos de regresion **no** se ven afectados por el escalamiento de features.

**Los de clasificacion si**

##  Las regresiones no necesitan escalamiento, mientras que las Clasificaciones si lo necesitan

# Simplificar las transformaciones con Pipelines

Para hacer el codigo mas reproducible y para evitar tener que aplicar multiples veces una misma transformacion es recomendable utilizar *sklearn.pipeline.make_pipeline* que permite encadenar transformaciones a los modelos

In [12]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(), Lasso())

model_scaled.fit(X_train,y_train)

Pipeline(steps=[('standardscaler', StandardScaler()), ('lasso', Lasso())])

In [13]:
print(model_scaled.score(X_test,y_test))

0.8404343154757754


### Crear features automaticamente

In [14]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [17]:
from sklearn.preprocessing import PolynomialFeatures
transformer = PolynomialFeatures(2)#grado del polinomio a ocupar
transformer.fit(A)
transformer.transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

In [18]:
X.shape

(4385, 7)

In [19]:
#genera todos los polinomios posibles con todos los features posibles
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

#se generan un total de 36 features

(4385, 36)

In [20]:
model_poly = make_pipeline(PolynomialFeatures(2),
                          Lasso())
model_poly.fit(X_train,y_train)
model_poly.score(X_test,y_test)

C:\Users\chris\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.448e+18, tolerance: 1.001e+16
  model = cd_fast.enet_coordinate_descent(


-3.5705337391568435

# Crear features categoricas

Son todas aquellas que tienen valores de un conjunto finito ej. pais,genero,calificacion 

Un mapeo del tipo (Peru,Chile,Colombia,Venezuela) -> (1,2,3,4) tiene el problema de asignarle un orden a los valores posibles de la categoria. Este orden impacta de distintas maneras a los algoritmo de ML por ejemplo aquellos que dependen de la topologia de R^n y de la funcion de distancia entre puntos en este espacio, consideraran que ciertas categorias se enceuntran mas cercanas unas de otras siendo que esto es generado puramente por el encoder de manera artificial y no por los datos per se.

Para no introducir informacion falsa o erronea en nuestros modelos existen formas mas inteligentes de encodear nuestros datos

**Encoding one-hot**

Este encoding consite en asignar una columna a cada categoria y rellenarla con 0 y 1 de la siguiente manera:

In [22]:
d= pd.DataFrame([['Chile','Colombia','Peru','Venezuela'],['H','M','H','M']])
d =d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,H
1,Colombia,M
2,Peru,H
3,Venezuela,M


In [23]:
#por cada valor en las columnas, se crea otra nueva
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Peru,pais_Venezuela,genero_H,genero_M
0,1,0,0,0,1,0
1,0,1,0,0,0,1
2,0,0,1,0,1,0
3,0,0,0,1,0,1


Sklearn tambien ofrece un objeto OneHotEncoder pero es un poco mas dificil de utilizar, asi que por criterios de la clase se ha elegido pd.get_dummies.

Sin embargo el objeto de eklearn tiene una ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso

Cuantas columnas se generarian con un one-hot-encoding de nuestras features categoricas??

In [24]:
movies_obj = pd.read_csv('C:\\Users\\chris\\Documents\\Jupyter Notebook\\PredIngresosPeliculas\movies_obj.csv')
movies_obj.head(10)

,movie_title,color,language,country,genres,content_rating,plot_keywords,director_name,actor_1_name,actor_2_name,actor_3_name
0,Avatar,Color,English,USA,Action|Adventure|Fantasy|Sci-Fi,PG-13,avatar|future|marine|native|paraplegic,James Cameron,CCH Pounder,Joel David Moore,Wes Studi
1,Pirates of the Caribbean: At World's End,Color,English,USA,Action|Adventure|Fantasy,PG-13,goddess|marriage ceremony|marriage proposal|pi...,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport
2,Spectre,Color,English,UK,Action|Adventure|Thriller,PG-13,bomb|espionage|sequel|spy|terrorist,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman
3,The Dark Knight Rises,Color,English,USA,Action|Thriller,PG-13,deception|imprisonment|lawlessness|police offi...,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt
4,Star Wars: Episode VII - The Force Awakens ...,NaN,NaN,NaN,Documentary,NaN,NaN,Doug Walker,Doug Walker,Rob Walker,NaN
5,John Carter,Color,English,USA,Action|Adventure|Sci-Fi,PG-13,alien|american civil war|male nipple|mars|prin...,Andrew Stanton,Daryl Sabara,Samantha Morton,Polly Walker
6,Spider-Man 3,Color,English,USA,Action|Adventure|Romance,PG-13,sandman|spider man|symbiote|venom|villain,Sam Raimi,J.K. Simmons,James Franco,Kirsten Dunst
7,Tangled,Color,English,USA,Adventure|Animation|Comedy|Family|Fantasy|Musi...,PG,17th century|based on fairy tale|disney|flower...,Nathan Greno,Brad Garrett,Donna Murphy,M.C. Gainey
8,Avengers: Age of Ultron,Color,English,USA,Action|Adventure|Sci-Fi,PG-13,artificial intelligence|based on comic book|ca...,Joss Whedon,Chris Hemsworth,Robert Downey Jr.,Scarlett Johansson
9,Harry Potter and the Half-Blood Prince,Color,English,UK,Adventure|Family|Fantasy|Mystery,PG,blood|book|love|potion|professor,David Yates,Alan Rickman,Daniel Radcliffe,Rupert Grint


In [26]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features mas informativas son las del casting. Sin embargo haciendo un one-hot-encoding de estas estariamos aumentando la dimension por mas de 2000!!

## **Encoding Binario**

**Esta tecnica no forma parte del ejercicio, solo se explora la funcionalidad**

No forma parte de sklearn, es parte de otra libreria, sin embargo el autor creo una API compatible con Sklearn asi que no supone un reto el utilizarla

    Categorica -> Numero -> Binario ->Columnas
    
Se recomienda su uso cuando el one-hot-encoder genera miles de dimensiones

In [27]:
#Se debe instalar
!pip install category_encoders

In [28]:
categoricals =  pd.read_csv('C:\\Users\\chris\\Documents\\Jupyter Notebook\\PredIngresosPeliculas\\categoricals.csv').set_index('Unnamed: 0')
categoricals.head(2)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [29]:
categoricals = categoricals.reset_index(drop=True).fillna(0)
#se rellena con 0 porque son menos de 10 los espacios en blanco

In [40]:
X_binenc = pd.concat([X,categoricals],axis=1)
X_binenc.head(2)

,production_budget,title_year,aspect_ratio,duration,budget,imdb_score,gross,actor_1_name,director_name
0,425000000.0,2009.00000,1.780000,178.000000,2.370000e+08,7.9,7.605058e+08,CCH Pounder,James Cameron
1,306000000.0,2002.41877,2.151445,107.628545,3.825188e+07,7.1,4.823375e+07,Doug Walker,Doug Walker


In [41]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name','director_name'])

In [43]:
 encoder.fit_transform(X_binenc).shape

(4385, 29)

In [44]:
binenc = encoder.fit_transform(X_binenc)

In [45]:
Xb_train,Xb_test,y_train,y_test = train_test_split(binenc,y)

In [46]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])

In [49]:
model_binenc = Lasso()
model = Lasso()

In [51]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso()

In [52]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))

0.8732947397454953
0.8758673302355817


# Conocimiento experto

Una gran parte del diseno de las features pasa por un conocimiento especifico del dominio en el que se esta trabajando.
Por ejemplo para analizar una imagen, nuestro cerebro no se concentra en los millones de pixeles de la imagen si no que solo en algunos detalles importantes como el contorno de los objetos.

Durante un buen tiempo los sistemas de computer vision encodeaban features que traducian este conocimiento experto (contornos).
Una de las unicas formas de obtener este conocimiento de manera sistematica es ir a buscar en repositorios algun paper de machine learning (como Arxiv) y estudiar la investigacion que se ha hecho sobre el dominio especifico


## Mas datos de calidad

Se debe tratar de conseguir mas datos que sean encodeables en features de calidad.

Ej. ***Piramide de Maslow del Machine learning***

In [55]:
#se ocupa el dataset que contenga las ganancias de las peliculas
#durante su primer fin de semana de exhibicion, asi como la cantidad
#de cines en la que fue estrenada

cine= pd.read_csv('C:\\Users\\chris\\Documents\\Jupyter Notebook\\PredIngresosPeliculas\\X_opening.csv')
cine.head(10)

,production_budget,worldwide_gross,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,opening_gross,screens
0,425000000.0,2.783919e+09,2009.0,1.78,178.0,4834.0,237000000.0,7.9,77025481.0,3452.0
1,300000000.0,9.634204e+08,2007.0,2.35,169.0,48350.0,300000000.0,7.1,139802190.0,4362.0
2,300000000.0,8.796209e+08,2015.0,2.35,148.0,11700.0,245000000.0,6.8,70403148.0,3929.0
3,275000000.0,1.084439e+09,2012.0,2.35,164.0,106759.0,250000000.0,8.5,160887295.0,4404.0
4,275000000.0,2.600021e+08,2013.0,2.35,150.0,45757.0,215000000.0,6.5,29210849.0,3904.0
5,275000000.0,2.827781e+08,2012.0,2.35,132.0,1873.0,263700000.0,6.6,30180188.0,3749.0
6,260000000.0,5.865819e+08,2010.0,1.85,100.0,2036.0,260000000.0,7.8,48767052.0,3603.0
7,258000000.0,8.908753e+08,2007.0,2.35,156.0,46055.0,258000000.0,6.2,151116516.0,4252.0
8,258000000.0,8.908753e+08,2007.0,2.35,156.0,46055.0,258000000.0,6.2,151116516.0,4252.0
9,250000000.0,1.404706e+09,2015.0,2.35,141.0,92000.0,250000000.0,7.5,191271109.0,4276.0
